In [ ]:
%reload_ext lab_black
import sqlite3
import pandas as pd

In [ ]:
batch_name = "error_injection_timing_test"
con = sqlite3.connect(
    f"/home/jupyter/tf/models/batch_run/{batch_name}/batch_results.sqlite"
)
cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f'Result tables: {cur.fetchall()}')

In [ ]:
batch_config = pd.read_sql_query("SELECT * from batch_config", con)
strain_df = pd.read_sql_query("SELECT * from strain", con)
grain_df = pd.read_sql

In [ ]:
# Print columns

cur.execute("select * from strain")
next(zip(*cur.description))

In [ ]:
query = """
    SELECT 
        code_name, 
        epoch, 
        timetick, 
        y, 
        testset, 
        AVG(acc) as acc, 
        AVG(sse) as sse, 
        AVG(conditional_sse) as conditional_sse
    FROM strain
    GROUP BY
        code_name, 
        epoch,
        timetick,
        y,
        testset
    """

strain_df = pd.read_sql_query(query, con)


In [ ]:
query = """
    SELECT 
        code_name, 
        epoch, 
        timetick, 
        y, 
        testset, 
        AVG(acc) as acc, 
        AVG(sse) as sse, 
        AVG(conditional_sse) as conditional_sse
    FROM grain
    GROUP BY
        code_name, 
        epoch,
        timetick,
        y,
        testset
    """

grain_df = pd.read_sql_query(query, con)


In [ ]:
grain_df['wordness'] = 'nonword'
strain_df['wordness'] = 'word'

df = pd.concat([strain_df, grain_df])
mean_df = df.groupby(['code_name', 'epoch', 'timetick', 'y', 'wordness']).mean().reset_index()
mean_df = mean_df.merge(batch_config[['code_name', 'inject_error_ticks']], "left", on="code_name")

In [ ]:
df.columns

In [ ]:
import altair as alt

stress_contrast_df = mean_df.groupby(['inject_error_ticks', 'epoch', 'timetick', 'y']).mean().reset_index()

def stress_plot(variable_of_interest):
    """Plot Hi time stress vs. Low time stress in variable of interest"""

    timetick_selection = alt.selection_single(
        bind=alt.binding_range(min=0, max=12, step=1),
        fields=["timetick"],
        init={"timetick": 12},
        name="timetick",
    )

    return alt.Chart(stress_contrast_df).mark_line().encode(
        x='epoch:Q',
        y=f'{variable_of_interest}:Q',
        color='y:N',
        column='inject_error_ticks:N',
    ).add_selection(timetick_selection).transform_filter(timetick_selection)

stress_plot('acc')

In [ ]:
import json

with open('/home/jupyter/tf/models/batch_run/error_injection_timing_test/batch_config.json') as f:
    batch_cfg = json.load(f)

In [ ]:
cfg_df = pd.DataFrame()

for i, cfg in enumerate(batch_cfg):
    cfg_df = pd.concat(
        [cfg_df,
        pd.DataFrame(cfg['params'], index=[i])]
    )


In [ ]:
cfg_df

In [ ]:
for model_cfg in cfg
pd.DataFrame()